EDA



In [37]:
 import pandas as pd
 import matplotlib.pyplot as plt
 import seaborn as sns

In [38]:
data=pd.read_excel("first inten project.xlsx")

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [43]:
data['date of reservation'] = pd.to_datetime(
    data['date of reservation'],
    errors='coerce')
data['date of reservation'] = data['date of reservation'].fillna(pd.Timestamp('2018-02-28'))



In [44]:
data['day'] = data['date of reservation'].dt.day
data['Year'] = data['date of reservation'].dt.year
data['Month'] = data['date of reservation'].dt.month

In [ ]:
reservations_per_month = data.groupby('Month').size()
print(reservations_per_month)
reservations_per_year = data.groupby('Year').size()
print(reservations_per_year)


In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=data, x='Year', y='average price')
plt.title('Average Price per Year')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
sns.barplot(data=data, x='Month', y='average price')
plt.title('Average Price per Month')
plt.show()


In [ ]:
year_status = pd.crosstab(data['Year'], data['booking status'])

print(year_status)

In [ ]:
month_status = pd.crosstab(data['Month'], data['booking status'])

print(month_status)

In [50]:
data['Total_Guests'] = data['number of adults'] + data['number of children']
data['Total_Nights'] = data['number of weekend nights'] + data['number of week nights']

In [ ]:
plt.figure(figsize=(8,5))
sns.lineplot(data=data, x='Total_Guests', y='special requests', ci=None,)
plt.title('Total Guests vs Average Special Requests')
plt.show()

plt.figure(figsize=(8,5))
sns.lineplot(data=data, x='Total_Guests', y='booking status', ci=None,)
plt.title('Total Guests vs booking status')
plt.show()

plt.figure(figsize=(8,5))
sns.lineplot(data=data, x='Total_Nights', y='booking status', ci=None)
plt.title('Total nights vs booking status')
plt.show()



In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(data=data, x='market segment type', y='lead time')
plt.title('Lead Time Distribution by Market Segment')
plt.xlabel('Market Segment Type')
plt.ylabel('Lead Time (days)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='lead time', y='average price', data=data)

plt.title('Relationship between Lead Time and Average Price')
plt.xlabel('Lead Time (Days)')
plt.ylabel('Average Price')
plt.grid(True)
plt.show()


plt.figure(figsize=(12, 6))
sns.histplot(data=data, x='lead time', hue='booking status')
plt.title('Relationship between Lead Time and Booking Status')
plt.xlabel('Lead Time (Days)')
plt.ylabel('Booking status')
plt.grid(True)
plt.show()

In [ ]:
numeric_colss = data.select_dtypes(include=["int64","float64"]).columns
for col in numeric_colss:
    plt.figure(figsize=(8,4))
    sns.kdeplot(data=data, x=col, hue='booking status', shade=True)
    plt.title(f'{col} distribution by Booking Status')
    plt.show()

In [ ]:
categorical_cols = data.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    sns.countplot(data=data,x=col,hue='booking status')
    plt.show()


# **Data Cleaning**

In [ ]:
data.isnull().sum()



In [ ]:
data.duplicated().sum()

In [ ]:
#check for unlogical data
data.describe()

In [59]:
#make encoder for each column its best (make dictionary)
from sklearn.preprocessing import LabelEncoder



encoded_columns=["booking status","type of meal","market segment type","room type"]
label_encoders = {}


for col in encoded_columns:
    le=LabelEncoder()
    data[col]=le.fit_transform(data[col])
    label_encoders[col] = le

In [ ]:
for col in data.columns:
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=data[col], color='lightgreen')
    plt.title(f'Box Plot of {col}')
    plt.ylabel("Booking status")
    plt.tight_layout()
    plt.show()

In [61]:
# Capping lead_time at the 95th percentile
lead_time_upper_limit = data['lead time'].quantile(0.95)
data['lead time'] = data['lead time'].clip(upper=lead_time_upper_limit)


average_price_upper_limit = data['average price'].quantile(0.95)
data['average price'] = data['average price'].clip(upper=average_price_upper_limit)

In [ ]:
import numpy as np
plt.figure(figsize=(10, 6))
correlation = data.select_dtypes(include=[np.number]).corr()

sns.heatmap(correlation[['booking status']].sort_values(by='booking status', ascending=False),
            annot=True, cmap='RdYlGn', fmt=".2f")

plt.title('Which factors lead to Cancellation?')
plt.show()

In [63]:
data.shape

(36285, 22)

In [64]:
data = data.drop(columns=[
    'Booking_ID',
    'date of reservation'
])

In [65]:
data.shape

(36285, 20)

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

# Separate features (X) and target (y)
X = data.drop(columns=["booking status"])
y = data["booking status"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# Apply scaler only to the feature sets (X_train, X_test)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


**Modeling**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

knn = KNeighborsClassifier(n_neighbors=7,weights='distance')

knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

#1 means non-cancellation ##0 cancelled

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=42)

dt_model.fit(X_train_scaled, y_train)


y_pred_dt = dt_model.predict(X_test_scaled)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

**Save preprocessing and models**

In [71]:
import joblib

joblib.dump(label_encoders, "label_encoders.pkl")

clip_values = {
    "lead_time_upper": lead_time_upper_limit,
    "avg_price_upper": average_price_upper_limit
}

joblib.dump(clip_values, "clip_values.pkl")

joblib.dump(scaler, "scaler.pkl")

joblib.dump(rf_model, "rf_model.pkl")
joblib.dump(dt_model, "dt_model.pkl")
joblib.dump(knn, "knn.pkl")

['knn.pkl']